In [55]:
import pandas as pd
import numpy as np
import time
from datetime import datetime

In [56]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

---

#### Read reference data into dataframe

In [57]:
df = pd.read_csv('../reference_w_loc.csv')

In [58]:
df.head(3)

,id,description,duration,location,reported_at,shape,sighted_at,geocoded_latitude,geocoded_longitude,city,state,country
0,0,"Man repts. witnessing &quot;flash, followed by...",NaN,"Iowa City, IA",19951009,unknown,19951009,41.661256,-91.529911,Iowa City,Iowa,us
1,1,"Man on Hwy 43 SW of Milwaukee sees large, bri...",2 min.,"Milwaukee, WI",19951011,unknown,19951010,43.034993,-87.922497,Milwaukee,Wisconsin,us
2,2,Telephoned Report:CA woman visiting daughter w...,NaN,"Shelton, WA",19950103,unknown,19950101,47.215094,-123.100707,Shelton,Washington,us


#### There are 138 rows with no sighted_at time information. We replace this date with their respective reported_at information

In [59]:
df[df['sighted_at'] == 0].shape

(138, 12)

In [60]:
# replace dates without a sighted_at with their reported_at value
for index,row in df[df['sighted_at'] == 0].iterrows():
    df.ix[index, 'sighted_at'] = row['reported_at']

In [61]:
# as expected there are no more missing sighted_at values
df[df['sighted_at'] == 0].shape

(0, 12)

#### Extract year from sighted_at and add as column

In [98]:
# add column 'year'
df['year'] = 0

In [99]:
def custom_round(x, base=10):
    return int(base * round(float(x)/base))

In [100]:
for index, row in df.iterrows():
    row_date = str(row['sighted_at'])
    df.ix[index, 'year'] = custom_round(datetime.strptime(row_date, '%Y%m%d').year)

In [101]:
df.sample(5)

,id,description,duration,location,reported_at,shape,sighted_at,geocoded_latitude,geocoded_longitude,city,state,country,year
22654,36375,Fire balls in the sky in AZI saw a fire ball i...,1 hour +,"Gold Canyon, AZ",20060308,fireball,20060301,33.371071,-111.426185,San Tan Valley,Arizona,us,2010
29489,47235,2 UFOs (saucer type) following each other in t...,10-15 seconds,"Coral Gables, FL",20080302,disk,20080302,25.690552,-80.235009,Coral Gables,Florida,us,2010
22643,36357,exploding/imploding and fading stars in Florid...,60+sec,"North Port, FL",20060316,light,20020630,27.044224,-82.235925,North Port,Florida,us,2000
8913,13499,36 year old reports an object he saw with his ...,15 Minutes,"Tampa, FL",20010816,other,19701001,27.947759,-82.458444,Tampa,Florida,us,1970
13209,20871,Rainbow colored cloud that changed colors cons...,6 min.,"Poway, CA",20030824,other,19950915,32.962823,-117.035865,Poway,California,us,2000


---

#### Read in population csv

In [87]:
pop_df = pd.read_csv('pop_final.csv')

In [88]:
pop_df.sample(4)

,State,Year,Population
2268,Washington,1900.0,518103
1376,Rhode Island,1980.0,947154
1288,North Carolina,2000.0,8049313
897,Delaware,1990.0,666168


In [89]:
df.dtypes

id                      int64
description            object
duration               object
location               object
reported_at             int64
shape                  object
sighted_at              int64
geocoded_latitude     float64
geocoded_longitude    float64
city                   object
state                  object
country                object
year                    int64
dtype: object

In [90]:
pop_df.dtypes

State          object
Year          float64
Population     object
dtype: object

In [92]:
for index,row in pop_df.iterrows():
    year = row['Year']
    try:
        int(year)
    except:
        print year

nan


In [93]:
df.shape

(38511, 13)

In [114]:
df = df.merge(pop_df, left_on=['state','year'], right_on=['State', 'Year'], how='left').drop_duplicates().drop(['State','Year'], axis=1)

In [115]:
df.shape

(38511, 14)

At this point, we've added the population column

#### save df to pick / read df from pickle

In [116]:
# df.to_pickle('../pickle_files/df_ref_w_population.pkl')
# df = pd.read_pickle('../pickle_files/df_ref_w_population.pkl')